In [7]:
import pandas as pd
from random import randint

In [13]:
instance_weight = pd.Series([n for n in range(10)], name='weight')

In [14]:
series = pd.Series([randint(10,20) for _ in range(10)], name='f')

In [15]:
flags = pd.Series([randint(0,1) for _ in range(10)], name='gbf')

In [17]:
df = pd.concat([series,flags,instance_weight], sort=False, axis=1)

In [18]:
value_counts = df[df[flags.name] == 0][series.name].value_counts().sort_index()

In [19]:
value_counts

10    1
15    1
17    2
20    2
Name: f, dtype: int64

In [41]:
tt = df[df[flags.name] == 0].groupby(series.name).agg({instance_weight.name:'sum'})

In [47]:
value_counts = tt[instance_weight.name].sort_index()

In [49]:
value_counts

f
10     3
15     8
17     9
20    10
Name: weight, dtype: int64

In [29]:
total_bads = df[df[flags.name] == 0][instance_weight.name].values.sum()

In [32]:
bads_pct = .2

In [33]:
bin_bads = round(total_bads * bads_pct, 0)

In [34]:
bin_bads

6.0

In [50]:
cutoffs = []
acc_bads = 0
for i, v in value_counts.iteritems():
    print(i, v)
    if cutoffs == []:
        cutoffs.append(-float('inf'))
    if acc_bads >= bin_bads:
        cutoffs.append(i)
        acc_bads = 0
    acc_bads += v
if acc_bads >= bin_bads:
    cutoffs.append(i)
else:
    cutoffs[-1] = i

10 3
15 8
17 9
20 10


In [51]:
cutoffs

[-inf, 17, 20, 20]

In [52]:
res = pd.cut(df[series.name], cutoffs, duplicates='drop')

In [53]:
res

0    (-inf, 17.0]
1    (17.0, 20.0]
2    (-inf, 17.0]
3    (-inf, 17.0]
4    (17.0, 20.0]
5    (-inf, 17.0]
6    (17.0, 20.0]
7    (-inf, 17.0]
8    (-inf, 17.0]
9    (-inf, 17.0]
Name: f, dtype: category
Categories (2, interval[float64]): [(-inf, 17.0] < (17.0, 20.0]]

In [39]:
df[series.name]

0    16
1    19
2    17
3    10
4    20
5    16
6    20
7    17
8    15
9    14
Name: f, dtype: int64

In [62]:
df.head()

,f,gbf,weight,category
0,16,1,0,"(-inf, 17.0]"
1,19,1,1,"(17.0, 20.0]"
2,17,0,2,"(-inf, 17.0]"
3,10,0,3,"(-inf, 17.0]"
4,20,0,4,"(17.0, 20.0]"


In [63]:
df['weighted_good'] = df.apply(lambda x: x[flags.name] * x[instance_weight.name], axis=1)

In [66]:
stats = df.groupby('category').agg({instance_weight.name: 'sum', 'weighted_good': 'sum'})

In [70]:
stats['weighted_good'].sum()

15

In [72]:
series.shape[0]

10